### Conversational Retrieval Chain

<b>Goal:</b> Make your RAG pipeline interactive and memory-aware

<b>Key Concepts:</b>
- Conversational Retrieval
- Chat Memory
- Answer grounding
- Source attribution

In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [2]:
# Load environment variables
load_dotenv()

True

### Load the docs

In [3]:
# Directory containing PDFs
pdf_dir = "policies"

# Load all PDFs
loaders = [PyPDFLoader(os.path.join(pdf_dir, file)) for file in os.listdir(pdf_dir) if file.endswith(".pdf")]

# Load and combine documents
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"✅ Loaded {len(documents)} pages from {len(loaders)} documents:")
for file in os.listdir(pdf_dir):
    if file.endswith(".pdf"):
        print("-", file)

✅ Loaded 15 pages from 3 documents:
- Leave-Policy.pdf
- posh-policy.pdf
- Salary_Policy.pdf


### Splitting into chunks

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"✅ Split into {len(chunks)} text chunks.")

✅ Split into 37 text chunks.


Each “chunk” is like a mini paragraph the AI can understand and embed.
This prevents token overflow and improves retrieval precision.

### Create Vector Store (Chroma) 

In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Using a free open-source embedding model to avoid OpenAI key issues
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create or reuse a Chroma vector database
vectorstore = Chroma.from_documents(chunks, embedding=embeddings, persist_directory="chroma_db")

# Initialize retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
print("✅ Vector store created and retriever ready.")

C:\Users\saket.khopkar\AppData\Local\Temp\ipykernel_17436\1880215437.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


✅ Vector store created and retriever ready.


"k": 1 ensures we retrieve the most relevant chunk to avoid token overflow.<br>
If you set k=3, it retrieves 3 most similar chunks, but that increases prompt size.

### Set up LLM

In [6]:
llm = ChatGroq(
    model="llama-3.1-8b-instant", 
    api_key=os.getenv("GROQ_API_KEY")
)

print("✅ Connected to Groq LLM successfully.")

✅ Connected to Groq LLM successfully.


### Creating Conversational Memory

In [10]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)
print("✅ Conversation memory initialized.")

✅ Conversation memory initialized.


This memory stores past user–AI exchanges.

Without it, the system would treat every question as a new, unrelated query.

Think of it as the “chat log” behind your assistant’s responses.

### Building the Conversational Retrieval Chain

In [11]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True
)

print("✅ Conversational Retrieval Chain is ready.")

✅ Conversational Retrieval Chain is ready.


In [ ]:
print("\n💬 Conversational RAG Assistant Ready!")
print("Type 'exit' to quit.\n")

chat_history = []  # optional, if you want to keep record manually

while True:
    query = input("You: ").strip()
    if query.lower() in ("exit", "quit"):
        print("👋 Goodbye!")
        break

    # ✅ print user query explicitly (so it's visible in notebook output)
    print(f"\n🧑‍💻 You asked: {query}\n")

    try:
        # run retrieval chain
        result = qa_chain.invoke({"question": query})
        answer = result.get("answer", "(no answer found)")

        # save to chat history (optional)
        chat_history.append({"user": query, "assistant": answer})

        # ✅ print assistant's answer
        print("🤖 Assistant:", answer)

        # ✅ print sources (if available)
        if "source_documents" in result:
            print("\n📚 Sources:")
            for doc in result["source_documents"]:
                print(f"- {doc.metadata.get('source', 'Unknown file')}")
        print()

    except Exception as e:
        print(f"⚠️ Error: {e}")


💬 Conversational RAG Assistant Ready!
Type 'exit' to quit.



You:  What is Casual Leave



🧑‍💻 You asked: What is Casual Leave

🤖 Assistant: Casual Leave is a type of leave that an employee can take when they need a day off for personal or miscellaneous reasons, not related to illness or injury. It is a paid leave, but has certain limitations and rules. Casual Leave can be taken for a minimum of 0.5 days to a maximum of 3 days, and any remaining unused leave lapses at the end of the year.

📚 Sources:
- policies\Leave-Policy.pdf



You:  What is the step to register complaints



🧑‍💻 You asked: What is the step to register complaints

🤖 Assistant: You can register complaints through the following methods:

1. Email: Submit your complaint to hrteam@BBIL Systems.com.
2. In-person discussion: You can discuss your complaint with any member of the Internal Committee mentioned in the policy within 3 months of the occurrence of the act of Sexual Harassment.

📚 Sources:
- policies\posh-policy.pdf



You:  What is the salary document about?



🧑‍💻 You asked: What is the salary document about?

🤖 Assistant: The salary document appears to be a public version of a salary policy. It is likely a guide or a set of rules that outlines how salaries are structured, paid, and managed within an entity. The document is part of a larger document set, titled "Talent Management and Culture" with a code of GT-02 and a revision number of 03.

📚 Sources:
- policies\Salary_Policy.pdf

